In [2]:
# import necessary packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


In [ ]:
# install selenium
!pip install selenium

In [ ]:
# install chromium chromedriver
!apt install chromium-chromedriver

In [5]:
from selenium import webdriver
# Import Options from selenium.webdriver.chrome to configure ChromeDriver settings.
from selenium.webdriver.chrome.options import Options

# Create an Options object to configure ChromeDriver settings.
chrome_options = Options()
# Add the '--headless' argument to Options, allowing Chrome to run without a GUI (graphically user interface).
chrome_options.add_argument('--headless')
# Add the '--no-sandbox' argument to bypass the OS security model; typically required in container environments like Docker.
chrome_options.add_argument('--no-sandbox')
# Add the '--disable-dev-shm-usage' argument to prevent issues with limited shared memory in certain environments.
chrome_options.add_argument('--disable-dev-shm-usage')

# Instantiate a Chrome WebDriver with the specified options, allowing programmatic control of Chrome.
driver = webdriver.Chrome(options=chrome_options)

In [6]:
# define url and send get request
url = "https://edmtrain.com/los-angeles-ca"
# Use the Selenium WebDriver to navigate to the specified URL.
driver.get(url)

In [7]:
# retrieve the source code of the current webpage loaded in the driver
html_content = driver.page_source

In [ ]:
html_content

In [9]:
# close the browser and release all resources associated with the driver
driver.quit()

In [10]:
# create beautiful soup object
soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
print(soup.prettify())

In [12]:
# find all divs with class: eventTitle
container = soup.find_all('div', class_='eventTitle')

In [ ]:
container

In [14]:
# find event containers and save data to all_events_data_list

event_containers = soup.find_all('div', class_='eventContainer')

all_events_data = []
for event_container in event_containers:
    data = {
        'datestr': event_container['datestr'],
        'eventid': event_container['eventid'],
        'titlestr': event_container['titlestr'],
        'venue': event_container.find('span', class_='eventVenue').text.strip(),  # Remove leading/trailing whitespace
        'genres': [a.text.strip() for a in event_container.find_all('a', class_='genre')]
    }
    all_events_data.append(data)

print(all_events_data)


[{'datestr': 'Sun, Apr 28', 'eventid': '328843', 'titlestr': "Sunrise 'The Morning Afters': Kariwood", 'venue': 'Los Angeles (Warehouse)', 'genres': ['deep disco house', 'urban contemporary']}, {'datestr': 'Sun, Apr 28', 'eventid': '328867', 'titlestr': 'Electric Day Club: Diskreet, Amergosa, Tio Ruben, Cacho, Myrixx, Vinizio, KayTexs, MauiiWauii, EGO, WAV3', 'venue': 'The Little Friend', 'genres': ['deep house', 'disco house', 'house', 'tech house', 'tribal house']}, {'datestr': 'Sun, Apr 28', 'eventid': '329088', 'titlestr': 'Sunday Sessions LA (Vinyl Only)', 'venue': 'Apotheke LA', 'genres': []}, {'datestr': 'Sun, Apr 28', 'eventid': '329229', 'titlestr': 'Nightshift Afterhours: Steven Sanchez, Patrick Tha Great, Koque, Justin Vilhauer, Casper', 'venue': 'Los Angeles', 'genres': []}, {'datestr': 'Tue, Apr 30', 'eventid': '324113', 'titlestr': 'Space Taco x Anonymous Studios - Workshop Powered by Pioneer DJ & Ableton', 'venue': 'The Basement Pomona', 'genres': []}, {'datestr': 'Wed, 

In [ ]:
# save events data to a df and explode the genres
df = pd.DataFrame(all_events_data)
df = df.explode('genres')


print(df)

In [17]:
# save event data to csv file
df.to_csv('scraped_events.csv', index=False)